In [0]:
#Configuración de conexión JDBC
JDBC_CONFIG = {
    "hostname": "psql-dn-keycloak-restore.postgres.database.azure.com",
    "port": 5432,
    "database": "keycloak",
    "username": dbutils.secrets.get(scope='secret-storeview', key='username-keycloak-db'),
    "password": dbutils.secrets.get(scope='secret-storeview', key='password-keycloak-db'),
    "driver": "org.postgresql.Driver"
}

jdbcUrl = f"jdbc:postgresql://{JDBC_CONFIG['hostname']}:{JDBC_CONFIG['port']}/{JDBC_CONFIG['database']}?sslmode=require"
connectionProperties = {
    "user": JDBC_CONFIG["username"],
    "password": JDBC_CONFIG["password"],
    "driver": JDBC_CONFIG["driver"]
}

In [0]:
#Se importa los tipos de datos necesarios para definir el esquema del DataFrame
from pyspark.sql.types import StructType, StructField, StringType, BooleanType

# # Controlar los tipos de datos y evitar inferencias automáticas de Spark
# schema_dn_user = StructType([
#     StructField("parent_group", StringType(), True),
#     StructField("grupo", StringType(), True),
#     StructField("nombre", StringType(), True),
#     StructField("apellido", StringType(), True),
#     StructField("email", StringType(), True),
#     StructField("enabled", BooleanType(), True),
#     StructField("verified", BooleanType(), True)
# ])

In [0]:
# Leer la tabla "dn_user" desde PostgreSQL usando el esquema definido
# Para garantizar que los tipos de datos sean los correctos
df_dn_user_access = spark.read \
    .format("jdbc") \
    .option("url", jdbcUrl) \
    .option("dbtable", "dn_user_access") \
    .option("user", connectionProperties["user"]) \
    .option("password", connectionProperties["password"]) \
    .option("driver", connectionProperties["driver"]) \
    .load()

# Mostrar los primeros 10 registros para verificar la lectura
display(df_dn_user_access.limit(10))

In [0]:
# Ruta destino en Bronze
bronze_path = "/mnt/bronze/keycloak/dn_user_access"

# Eliminar en caso que exista
try:
  dbutils.fs.ls(bronze_path)
  print("Existe, se sobreescribirá")
except:
  print("No existe, se creará")

# Escribir en delta
df_dn_user_access.write.format("delta").mode("overwrite").save(bronze_path)

# Registrar tabla
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS bronze.dn_user_access
    USING DELTA
    LOCATION '{bronze_path}/'
""")
spark.sql("REFRESH TABLE bronze.dn_user_access")

In [0]:
%sql
SELECT * FROM bronze.dn_user_access
LIMIT 100

In [0]:
%sql
DESCRIBE bronze.dn_user_access